# TSA Chapter 2: Lag Operator and Difference Operators

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_Ch2/TSA_ch2_lag_operator/TSA_ch2_lag_operator.ipynb)

This notebook demonstrates:
- Lag operator L, difference operator Delta, and lag polynomial notation for AR, MA, and ARMA models
- Numerical demonstration of first and second differences

In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from matplotlib.patches import Polygon
# Style configuration
COLORS = {
    'blue': '#1A3A6E',
    'red': '#DC3545',
    'green': '#2E7D32',
    'orange': '#E67E22',
    'gray': '#666666',
    'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none',
    'figure.facecolor': 'none',
    'savefig.transparent': True,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': False,
    'font.size': 9,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'legend.fontsize': 8,
    'figure.dpi': 150,
    'lines.linewidth': 1.2,
    'axes.edgecolor': '#333333',
    'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    """Save chart as PDF and PNG."""
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}.pdf + .png')

In [ ]:
# Set random seed

n = 100
t = np.arange(n)

# Generate a sample time series (random walk with trend)
trend = 0.5 * t
noise = np.cumsum(np.random.normal(0, 1, n))
X = 100 + trend + noise

print("=" * 60)
print("THE LAG OPERATOR (BACKSHIFT OPERATOR)")
print("=" * 60)

print("""
Definition: L (or B) is the lag operator
  L X_t = X_{t-1}
  L^k X_t = X_{t-k}

Key Properties:
  L^0 = 1 (identity)
  L^a × L^b = L^{a+b}
  (L + L^2) X_t = X_{t-1} + X_{t-2}

Difference Operator:
  Δ = 1 - L
  ΔX_t = (1-L)X_t = X_t - X_{t-1}

Second Difference:
  Δ² = (1-L)² = 1 - 2L + L²
  Δ²X_t = X_t - 2X_{t-1} + X_{t-2}
""")

# Demonstrate operations
print("\n" + "=" * 60)
print("NUMERICAL EXAMPLE")
print("=" * 60)
print(f"\nOriginal series X: {X[:5].round(2)}")
print(f"                   X_1={X[0]:.2f}, X_2={X[1]:.2f}, X_3={X[2]:.2f}, ...")

# First difference
diff1 = np.diff(X)
print(f"\nFirst difference ΔX = (1-L)X:")
print(f"  ΔX_2 = X_2 - X_1 = {X[1]:.2f} - {X[0]:.2f} = {diff1[0]:.2f}")
print(f"  ΔX_3 = X_3 - X_2 = {X[2]:.2f} - {X[1]:.2f} = {diff1[1]:.2f}")

# Second difference
diff2 = np.diff(diff1)
print(f"\nSecond difference Δ²X = (1-L)²X:")
print(f"  Δ²X_3 = X_3 - 2X_2 + X_1")
print(f"        = {X[2]:.2f} - 2×{X[1]:.2f} + {X[0]:.2f}")
print(f"        = {X[2] - 2*X[1] + X[0]:.2f}")
print(f"  Verify: {diff2[0]:.2f}")

# Create visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Original series
ax1 = axes[0, 0]
ax1.plot(X, 'b-', linewidth=1)
ax1.set_title('Original Series $X_t$', fontsize=12)
ax1.set_xlabel('Time')
ax1.set_ylabel('Value')
ax1.grid(True, alpha=0.3)

# First difference
ax2 = axes[0, 1]
ax2.plot(diff1, 'g-', linewidth=1)
ax2.axhline(y=0, color='red', linestyle='--')
ax2.axhline(y=np.mean(diff1), color='orange', linestyle='-', linewidth=2, label=f'Mean = {np.mean(diff1):.2f}')
ax2.set_title('First Difference $\\Delta X_t = (1-L)X_t$', fontsize=12)
ax2.set_xlabel('Time')
ax2.set_ylabel('Value')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Second difference
ax3 = axes[1, 0]
ax3.plot(diff2, 'r-', linewidth=1)
ax3.axhline(y=0, color='black', linestyle='--')
ax3.set_title('Second Difference $\\Delta^2 X_t = (1-L)^2 X_t$', fontsize=12)
ax3.set_xlabel('Time')
ax3.set_ylabel('Value')
ax3.grid(True, alpha=0.3)

# Summary
ax4 = axes[1, 1]
ax4.axis('off')
summary = """
Lag Operator Summary
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Notation:
  L X_t = X_{t-1}     (one lag)
  L^k X_t = X_{t-k}   (k lags)

Difference Operator:
  Δ = (1 - L)
  ΔX_t = X_t - X_{t-1}

Second Difference:
  Δ² = (1-L)² = 1 - 2L + L²
  Δ²X_t = X_t - 2X_{t-1} + X_{t-2}

AR(p) in Lag Notation:
  φ(L)X_t = ε_t
  where φ(L) = 1 - φ₁L - φ₂L² - ... - φₚLᵖ

MA(q) in Lag Notation:
  X_t = θ(L)ε_t
  where θ(L) = 1 + θ₁L + θ₂L² + ... + θqLq

ARMA(p,q):
  φ(L)X_t = θ(L)ε_t
"""
ax4.text(0.05, 0.95, summary, transform=ax4.transAxes, fontsize=10,
         verticalalignment='top', family='monospace',
         bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.5))

plt.tight_layout()
save_chart(fig, 'ch2_lag_operator')
plt.show()

print("\n" + "=" * 60)
print("WHY LAG NOTATION IS USEFUL")
print("=" * 60)
print("""
1. Compact representation of complex models
2. Easy manipulation using algebra
3. Roots of lag polynomials determine stationarity/invertibility
4. Facilitates forecasting formulas
""")